In [1]:
import numpy as np
import h5py

import GRANDhdf5DataRunLevel as DataRun
import GRANDhdf5DataEventLevel as DataEvent
import GRANDhdf5Utilities as Util

In [2]:
file_name="/mnt/external/Pesquisa/GRAND/HDF5_tests/Charles/Run22_2020_11_09.hdf5"
input_hdf5= h5py.File(file_name,'r')

In [3]:
run_list = list(input_hdf5.keys())

In [4]:
run = input_hdf5[run_list[0]]
run.name

'/Run_22'

# # Testing the file structure

In [5]:
for name in run:
    print(name)

CenterField
DetectorInfo
ElectronicsSettings
Event_1594
Event_1698
Event_446
Event_4718
Event_538
Event_5536
Event_601
Event_6353
Event_6448
Event_6589
Event_6755
Event_6824
Event_7296
Event_7651
Event_789
Event_8038
Event_98
Monitor


In [6]:
run.keys()

<KeysViewHDF5 ['CenterField', 'DetectorInfo', 'ElectronicsSettings', 'Event_1594', 'Event_1698', 'Event_446', 'Event_4718', 'Event_538', 'Event_5536', 'Event_601', 'Event_6353', 'Event_6448', 'Event_6589', 'Event_6755', 'Event_6824', 'Event_7296', 'Event_7651', 'Event_789', 'Event_8038', 'Event_98', 'Monitor']>

In [7]:
run['CenterField']

<HDF5 dataset "CenterField": shape (1,), type "|V200">

In [8]:
run['CenterField'][()].dtype.names

('center_latitude',
 'center_longitude',
 'center_altitude',
 'center_x',
 'center_y')

In [9]:
run['DetectorInfo'][()]

array([(1, 38.852256 , 92.3454847, 2712.06, -46.784622  , -60.517822, b'GP35', 127, b'AERA'),
       (2, 38.8524178, 92.3441787, 2712.11, -28.796738  ,  52.55267 , b'GP35', 105, b'AERA'),
       (3, 38.8533595, 92.3446684, 2711.16,  75.895424  ,  10.155565, b'GP35', 126, b'AERA'),
       (4, 38.852674 , 92.344811 , 2712.  ,  -0.31406537,  -2.190416, b'GP35',   0, b'R&S SCOPE')],
      dtype={'names':['run_antenna_id','run_latitude','run_longitude','run_altitude','run_x','run_y','run_antenna_model','run_electronics_id','run_electronics_model'], 'formats':['<u2','<f8','<f8','<f4','<f4','<f4','S20','<u2','S20'], 'offsets':[0,16,8,24,28,32,36,56,58], 'itemsize':200})

In [10]:
def printname(name):
    print(name)
    
run.visit(printname)

CenterField
DetectorInfo
ElectronicsSettings
Event_1594
Event_1594/AntennaInfo
Event_1594/EventHeader
Event_1594/Traces_1
Event_1594/Traces_1/ADC_X
Event_1594/Traces_1/ADC_Y
Event_1594/Traces_1/ADC_Z
Event_1594/Traces_2
Event_1594/Traces_2/ADC_X
Event_1594/Traces_2/ADC_Y
Event_1594/Traces_2/ADC_Z
Event_1594/Traces_3
Event_1594/Traces_3/ADC_X
Event_1594/Traces_3/ADC_Y
Event_1594/Traces_3/ADC_Z
Event_1698
Event_1698/AntennaInfo
Event_1698/EventHeader
Event_1698/Traces_1
Event_1698/Traces_1/ADC_X
Event_1698/Traces_1/ADC_Y
Event_1698/Traces_1/ADC_Z
Event_1698/Traces_2
Event_1698/Traces_2/ADC_X
Event_1698/Traces_2/ADC_Y
Event_1698/Traces_2/ADC_Z
Event_1698/Traces_3
Event_1698/Traces_3/ADC_X
Event_1698/Traces_3/ADC_Y
Event_1698/Traces_3/ADC_Z
Event_446
Event_446/AntennaInfo
Event_446/EventHeader
Event_446/Traces_1
Event_446/Traces_1/ADC_X
Event_446/Traces_1/ADC_Y
Event_446/Traces_1/ADC_Z
Event_446/Traces_2
Event_446/Traces_2/ADC_X
Event_446/Traces_2/ADC_Y
Event_446/Traces_2/ADC_Z
Event_446/T

# Write to a file using Matias' methods

In [11]:
output_hdf5 = h5py.File('Bruno_output.hdf5','a')

In [12]:
# Define run level records
center_field_input = np.zeros(1,DataRun.CenterField_dtype)
detector_info_input = np.zeros(1,DataRun.DetectorInfo_dtype)
electronics_settings_input = np.zeros(1,DataRun.ElectronicsSettings_dtype)

In [13]:
# Define event level records
antenna_info_input = np.zeros(1,DataEvent.AntennaInfo_dtype)
event_header_input = np.zeros(1,DataEvent.EventHeader_dtype)

In [14]:
# Fill run level variables
RunID = run.name.split('_')[1]
for field in run['CenterField'][()].dtype.names:
    center_field_input[field] = run['CenterField'][()][field]
    
center_field_data = DataRun.DataRunAddCenterField(output_hdf5, RunID, center_field_input)

In [15]:
for detector in range(len(run['DetectorInfo'][()])):
    for field in run['DetectorInfo'][()][detector].dtype.names:
        detector_info_input[field] = run['DetectorInfo'][()][detector][field]
    detector_info_data = DataRun.DataRunAddDetectorInfo(output_hdf5, RunID, detector_info_input)

In [16]:
for elec in range(len(run['ElectronicsSettings'][()])):
    for field in run['ElectronicsSettings'][()][elec].dtype.names:
        electronics_settings_input[field] = run['ElectronicsSettings'][()][elec][field]
    electronics_settings_date = DataRun.DataRunAddElectronicsSettings(output_hdf5, RunID, electronics_settings_input)

In [17]:
for evt_name, evt_dataset in run.items():
    if evt_name.startswith('Event'):
        for key, dataset in evt_dataset.items():
            if key == 'AntennaInfo':
                for antenna in range(len(dataset[()])):
                    for field in dataset[()][antenna].dtype.names:
                        antenna_info_input[field] = dataset[()][antenna][field]
                    EventID = evt_name.split('_')[1]
                    antenna_info_data = DataEvent.DataEventAddAntennaInfo(output_hdf5, RunID, EventID, antenna_info_input)
            
            if key == 'EventHeader':
                for field in dataset[()].dtype.names:
                    event_header_input[field] = dataset[()][field]
                EventID = evt_name.split('_')[1]
                event_header_data = DataEvent.DataEventAddEventHeader(output_hdf5, RunID, EventID, event_header_input)
            
            if key.startswith("Traces"):
                TraceX = dataset['ADC_X'][()]
                TraceY = dataset['ADC_Y'][()]
                TraceZ = dataset['ADC_Z'][()]
                EventID = evt_name.split('_')[1]
                AntennaID = key.split("_",1)[1]
                DataEvent.DataEventAddTraces(output_hdf5, RunID, EventID, AntennaID,TraceX,TraceY,TraceZ)

In [18]:
output_hdf5.close()